![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [257]:
import pandas as pd
import numpy as np

# Start coding here...

In [258]:
df = pd.read_csv('bank_marketing.csv')

df.head(3)

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,13,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,19,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,23,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [259]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   client_id       41188 non-null  int64  
 1   age             41188 non-null  int64  
 2   job             41188 non-null  object 
 3   marital         41188 non-null  object 
 4   education       41188 non-null  object 
 5   credit_default  41188 non-null  object 
 6   housing         41188 non-null  object 
 7   loan            41188 non-null  object 
 8   contact         41188 non-null  object 
 9   month           41188 non-null  object 
 10  day             41188 non-null  int64  
 11  duration        41188 non-null  int64  
 12  campaign        41188 non-null  int64  
 13  pdays           41188 non-null  int64  
 14  previous        41188 non-null  int64  
 15  poutcome        41188 non-null  object 
 16  emp_var_rate    41188 non-null  float64
 17  cons_price_idx  41188 non-null 

In [260]:
client_df = df[['client_id', 'age', 'job', 'marital' , 'education' , 'credit_default', 'housing', 'loan']]


In [261]:
client_df.head(3)

,client_id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no


In [262]:
client_df.rename(columns={'client_id' : 'id'}, inplace=True)

client_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       41188 non-null  object
 5   credit_default  41188 non-null  object
 6   housing         41188 non-null  object
 7   loan            41188 non-null  object
dtypes: int64(2), object(6)
memory usage: 2.5+ MB


In [263]:
from datetime import datetime

currentYear = datetime.now().year

print(currentYear)

2023


In [264]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   client_id       41188 non-null  int64  
 1   age             41188 non-null  int64  
 2   job             41188 non-null  object 
 3   marital         41188 non-null  object 
 4   education       41188 non-null  object 
 5   credit_default  41188 non-null  object 
 6   housing         41188 non-null  object 
 7   loan            41188 non-null  object 
 8   contact         41188 non-null  object 
 9   month           41188 non-null  object 
 10  day             41188 non-null  int64  
 11  duration        41188 non-null  int64  
 12  campaign        41188 non-null  int64  
 13  pdays           41188 non-null  int64  
 14  previous        41188 non-null  int64  
 15  poutcome        41188 non-null  object 
 16  emp_var_rate    41188 non-null  float64
 17  cons_price_idx  41188 non-null 

In [265]:
df[['day', 'month']].head(10)

,day,month
0,13,may
1,19,may
2,23,may
3,27,may
4,3,may
5,5,may
6,3,may
7,12,may
8,21,may
9,5,may


In [266]:
campaign_df=df[['client_id','campaign','duration', 'pdays', 'previous', 'poutcome', 'y']]

In [267]:
campaign_df.head(10)

,client_id,campaign,duration,pdays,previous,poutcome,y
0,0,1,261,999,0,nonexistent,no
1,1,1,149,999,0,nonexistent,no
2,2,1,226,999,0,nonexistent,no
3,3,1,151,999,0,nonexistent,no
4,4,1,307,999,0,nonexistent,no
5,5,1,198,999,0,nonexistent,no
6,6,1,139,999,0,nonexistent,no
7,7,1,217,999,0,nonexistent,no
8,8,1,380,999,0,nonexistent,no
9,9,1,50,999,0,nonexistent,no


In [268]:
campaign_df = campaign_df.assign(last_contact_date=2022).astype(str)

In [269]:
campaign_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   client_id          41188 non-null  object
 1   campaign           41188 non-null  object
 2   duration           41188 non-null  object
 3   pdays              41188 non-null  object
 4   previous           41188 non-null  object
 5   poutcome           41188 non-null  object
 6   y                  41188 non-null  object
 7   last_contact_date  41188 non-null  object
dtypes: object(8)
memory usage: 2.5+ MB


In [270]:
campaign_df.head(3)

,client_id,campaign,duration,pdays,previous,poutcome,y,last_contact_date
0,0,1,261,999,0,nonexistent,no,2022
1,1,1,149,999,0,nonexistent,no,2022
2,2,1,226,999,0,nonexistent,no,2022


In [271]:
campaign_df['last_contact_date'] = campaign_df['last_contact_date'].str.cat(df['month'], 
                                                                            sep ="-")

In [272]:
campaign_df['last_contact_date'] = campaign_df['last_contact_date'].str.cat(df['day'].astype(str), sep ="-")

In [273]:
campaign_df.head(3)

,client_id,campaign,duration,pdays,previous,poutcome,y,last_contact_date
0,0,1,261,999,0,nonexistent,no,2022-may-13
1,1,1,149,999,0,nonexistent,no,2022-may-19
2,2,1,226,999,0,nonexistent,no,2022-may-23


In [274]:
economics_df = df[['client_id', 'emp_var_rate', 'cons_price_idx','euribor3m','nr_employed']]

In [275]:
campaign_dict={'duration' : 'contact_duration','previous' :'previous_campaign_contacts','y' : 'campaign_outcome','poutcome' : 'previous_outcome','campaign' : 'number_contacts'}

In [276]:
campaign_df.head(3)

,client_id,campaign,duration,pdays,previous,poutcome,y,last_contact_date
0,0,1,261,999,0,nonexistent,no,2022-may-13
1,1,1,149,999,0,nonexistent,no,2022-may-19
2,2,1,226,999,0,nonexistent,no,2022-may-23


In [277]:
campaign_df.rename(columns=campaign_dict, inplace = True)


In [278]:
campaign_df['campaign_id'] = 1

In [279]:
campaign_df.head(5)

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date,campaign_id
0,0,1,261,999,0,nonexistent,no,2022-may-13,1
1,1,1,149,999,0,nonexistent,no,2022-may-19,1
2,2,1,226,999,0,nonexistent,no,2022-may-23,1
3,3,1,151,999,0,nonexistent,no,2022-may-27,1
4,4,1,307,999,0,nonexistent,no,2022-may-3,1


In [280]:
economics_df.rename(columns={'euribor3m' : 'euribor_three_months',
                            'nr_employed' : 'number_employed'}, inplace= True)

In [281]:
client_df.head(10)

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes
5,5,45,services,married,basic.9y,unknown,no,no
6,6,59,admin.,married,professional.course,no,no,no
7,7,41,blue-collar,married,unknown,unknown,no,no
8,8,24,technician,single,professional.course,no,yes,no
9,9,25,services,single,high.school,no,yes,no


In [282]:
client_df['education'] = client_df['education'].apply(lambda x: x.replace('.', '_'))

In [283]:
client_df.head(10)

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin.,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes
5,5,45,services,married,basic_9y,unknown,no,no
6,6,59,admin.,married,professional_course,no,no,no
7,7,41,blue-collar,married,unknown,unknown,no,no
8,8,24,technician,single,professional_course,no,yes,no
9,9,25,services,single,high_school,no,yes,no


In [284]:
client_df[['education']].groupby(['education']).count()

""
education
basic_4y
basic_6y
basic_9y
high_school
illiterate
professional_course
university_degree
unknown


In [285]:
client_df['education'] = client_df['education'].replace('unknown', np.nan)

In [286]:
client_df.head(10)

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin.,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes
5,5,45,services,married,basic_9y,unknown,no,no
6,6,59,admin.,married,professional_course,no,no,no
7,7,41,blue-collar,married,NaN,unknown,no,no
8,8,24,technician,single,professional_course,no,yes,no
9,9,25,services,single,high_school,no,yes,no


In [287]:
client_df['job'] = client_df['job'].apply(lambda x: x.replace('.', ''))

In [288]:
client_df.head(10)

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes
5,5,45,services,married,basic_9y,unknown,no,no
6,6,59,admin,married,professional_course,no,no,no
7,7,41,blue-collar,married,NaN,unknown,no,no
8,8,24,technician,single,professional_course,no,yes,no
9,9,25,services,single,high_school,no,yes,no


In [289]:
campaign_df.head(10)

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date,campaign_id
0,0,1,261,999,0,nonexistent,no,2022-may-13,1
1,1,1,149,999,0,nonexistent,no,2022-may-19,1
2,2,1,226,999,0,nonexistent,no,2022-may-23,1
3,3,1,151,999,0,nonexistent,no,2022-may-27,1
4,4,1,307,999,0,nonexistent,no,2022-may-3,1
5,5,1,198,999,0,nonexistent,no,2022-may-5,1
6,6,1,139,999,0,nonexistent,no,2022-may-3,1
7,7,1,217,999,0,nonexistent,no,2022-may-12,1
8,8,1,380,999,0,nonexistent,no,2022-may-21,1
9,9,1,50,999,0,nonexistent,no,2022-may-5,1


In [290]:
campaign_df[['previous_outcome']].groupby(['previous_outcome']).count()

""
previous_outcome
failure
nonexistent
success


In [291]:
campaign_df['previous_outcome'] = campaign_df['previous_outcome'].apply(lambda x: x.replace('success', '1'))

In [292]:
campaign_df['previous_outcome'] = campaign_df['previous_outcome'].apply(lambda x: x.replace('failure', '0'))

In [293]:
campaign_df['previous_outcome'] = campaign_df['previous_outcome'].replace('nonexistent', np.nan)

In [294]:
campaign_df[['campaign_outcome']].groupby(['campaign_outcome']).count()

""
campaign_outcome
no
yes


In [295]:
campaign_df['campaign_outcome'] = campaign_df['campaign_outcome'].apply(lambda x: x.replace('yes', '1'))

In [296]:
campaign_df['campaign_outcome'] = campaign_df['campaign_outcome'].apply(lambda x: x.replace('no', '0'))

In [297]:
client_df.to_csv('client.csv', index=False)

In [298]:
campaign_df.to_csv('campaign.csv', index=False)

In [299]:
economics_df.to_csv('economics.csv', index=False)

In [300]:
client_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       39457 non-null  object
 5   credit_default  41188 non-null  object
 6   housing         41188 non-null  object
 7   loan            41188 non-null  object
dtypes: int64(2), object(6)
memory usage: 2.5+ MB


In [301]:
client_table = """CREATE TABLE client
(
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

In [302]:
campaign_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   client_id                   41188 non-null  object
 1   number_contacts             41188 non-null  object
 2   contact_duration            41188 non-null  object
 3   pdays                       41188 non-null  object
 4   previous_campaign_contacts  41188 non-null  object
 5   previous_outcome            5625 non-null   object
 6   campaign_outcome            41188 non-null  object
 7   last_contact_date           41188 non-null  object
 8   campaign_id                 41188 non-null  int64 
dtypes: int64(1), object(8)
memory usage: 2.8+ MB


In [303]:
campaign_table = """CREATE TABLE campaign
(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE    
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""

In [304]:
economics_table = """CREATE TABLE economics
(
    client_id SERIAL references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""